# Principle of autograph

In the notebook, we try to understand how @tf.function works.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
@tf.function(autograph=True)
def myadd(a,b):
    for i in tf.range(3):
        tf.print(i)
    c = a+b
    print("tracing")
    return c

Nothing happens until here. Python just adds the new function to its stack.

We use this function for the first time.

In [3]:
myadd(tf.constant("hello"),tf.constant("world"))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

There are prints from the for loop and the print of "tracing" at the end of function. However, the order is not the same as defined in the function.

The first thing @tf.function when it's called is to create a static graph. 

To create a static graph, Tensorflow runs the python code to ensure the data type for tensor and generate operator. 

In this process, @tf.function tries to replace for, while, if, break, continure , return with tf function. So the tf.print is not excuted at the beginning, while the print("tracing") is excuted.

When the static graph is constructed, the tensorflow code is excuted. The printing in the loop are shown in the second time.

In [4]:
myadd(tf.constant("hello"),tf.constant("world"))

0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

Rerun the same function, there is no printing "tracing". Because the static model is already there.

Next I define a pure python function without tf function. Find what happens in the first call and second call.

In [8]:
@tf.function(autograph=True)
def mynewadd(a,b):
    for i in range(3):
        print(i)
    c = a+b
    print("tracing")
    return c

In [9]:
mynewadd(tf.constant("hello"),tf.constant("world"))

0
1
2
tracing


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

In [10]:
mynewadd(tf.constant("hello"),tf.constant("world"))

<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

Compared to the last example, we find the printing in the same order as defined in function for the first call. For the second call, there is no printing.  Because python print is neither the code to define data type nor the code to define the operator

In [11]:
mynewadd(tf.constant("hellooooo"),tf.constant("world!"))

<tf.Tensor: shape=(), dtype=string, numpy=b'helloooooworld!'>

The input value is changed, while the data type stays the same. The constructed static graph is compatible. What about changing the data type of input value.

In [12]:
mynewadd(tf.constant(1.0),tf.constant(2.0))

0
1
2
tracing


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

The data type is changed to tf.float32 from string. The data type of old static graph is not suitable to process numerical data. So Tensorflow has to reconstruct the static graph. What about feeding tf.int32 to this new graph.

In [13]:
mynewadd(tf.constant(1),tf.constant(2))

0
1
2
tracing


<tf.Tensor: shape=(), dtype=int32, numpy=3>

New static graph is constructed for tf.int32 data.

### Rethink the suggestions for autograph

1. Use tf supported functions. Replace print by tf.print, because print is not included in the static graph.

2. Avoid defining tf.Variable. tf.Variable is considered and only considered during the generating of static function. This line of code has no effect during the runtime of a static graph. Luckily, the interpretor gives error when detecting tf.Variable in @tf.function

3. The static graph is built in C++, the python external list, dict, etc. can't be integrated. They are only scanned during the constructing of static graph.